In [615]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

import flask
from flask_cors import CORS

import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

import os
from os import environ
import requests
import io

import datetime
from dateutil.relativedelta import relativedelta

import socket

import seaborn as sns
from itertools import groupby

In [616]:
Data=pd.read_csv('SQLsnippet_OPO.csv')
Data.columns=[x.replace(' ','') for x in Data.columns]

In [617]:
Data['min_stay']=Data[['is_one_way','min_stay']].apply(lambda x: -1 if x[0]==1 else x[1],axis=1)
minstay_map=DataFrame([[x for x in range(-1,11)],['one way','0 days','1 day']+[str(x)+' days' for x in range(2,11)]],index=['min_stay','min_stay_new']).transpose()
Data=Data.merge(minstay_map,on='min_stay')
Data['min_stay']=Data['min_stay_new']
Data=Data[[x for x in Data.columns if x!='min_stay_new']]

In [618]:
Data_oneway=Data.loc[Data['min_stay']=='one way',['carrier','observation_date','origin','destination','outbound_departure_date','min_stay',
'outbound_departure_time','outbound_seats','price_outbound','price_exc']]
Data_oneway['price_exc']=Data_oneway[['price_outbound','price_exc']].apply(lambda x: x[0] if pd.isnull(x[1]) else x[1],axis=1)
Data_oneway=Data_oneway[[x for x in Data_oneway.columns if x!='price_outbound']]

Data_twoway=Data.loc[Data['min_stay']!='one way',['carrier','observation_date','origin','destination','min_stay','outbound_departure_date',
'outbound_departure_time','outbound_seats','inbound_departure_date','inbound_departure_time','inbound_seats','price_outbound','price_inbound','price_exc']]

In [ ]:
Data_oneway_min=Data_oneway.groupby(['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time'])['price_exc']\
.min().reset_index().rename(columns={'price_exc': 'price_outbound_min'})
Data_twoway_min=Data_twoway.groupby(['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time'])['price_outbound','price_inbound','price_exc']\
.min().reset_index().rename(columns={'price_outbound': 'price_outbound_min','price_inbound': 'price_inbound_min','price_exc': 'price_exc_min'})

Data_twoway_mincomb=Data_twoway.merge(Data_twoway_min[['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','price_exc_min','price_outbound_min',
                                                                'price_inbound_min']],
on=['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time'])

Data_twoway_mincomb=Data_twoway_mincomb.loc[Data_twoway_mincomb['price_exc']==Data_twoway_mincomb['price_exc_min'],
                                                 [x for x in Data_twoway_mincomb.columns if '_min' not in x]]

Data_twoway_mincomb=Data_twoway_mincomb.rename(columns={'price_outbound': 'price_outbound_mincomb', 'price_inbound': 'price_inbound_mincomb'})
Data_twoway_mincomb=Data_twoway_mincomb[[x for x in Data_twoway_mincomb.columns if 'price_exc' not in x]]

Data_twoway_min=Data_twoway_min.merge(Data_twoway_mincomb[['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time','price_outbound_mincomb','price_inbound_mincomb']],
on=['carrier','origin','destination','outbound_departure_date','min_stay','outbound_departure_time'])

In [ ]:
Data_oneway_min=Data_oneway_min.drop_duplicates()
Data_twoway_min=Data_twoway_min.drop_duplicates()

In [ ]:
Data_oneway_min['price_exc_min']=Data_oneway_min['price_outbound_min']

In [ ]:
Data_twoway_min=Data_twoway_min[[x for x in Data_twoway_min if 'comb' not in x]]

In [ ]:
Data=pd.concat([Data_oneway_min,Data_twoway_min])

C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




# Dash

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [ ]:
intermediate=Data[['origin','destination','outbound_departure_time','min_stay']].drop_duplicates()

In [ ]:
Data['outbound_departure_date']=pd.to_datetime(Data['outbound_departure_date'])

In [ ]:
checkbox_values=intermediate\
.sort_values(by=['origin','destination','outbound_departure_time','min_stay']).transpose().values.tolist()

In [ ]:
PAGE_SIZE = 20

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        html.Div([html.Div(style={'width': '10%', 'display': 'inline-block'}),
        html.Img(src=app.get_asset_url('LuxairGroup.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle', 
                       'width': '10%', 'height': '7em'}),
        html.H4(children='Price Monitoring System',style={
            'textAlign': 'center',
            'color': 'black','height': '1em', 'display': 'inline-block', 'width': '64%'}),
            html.Img(src=app.get_asset_url('Luxair.jpg'), 
              style = {'backgroundColor' : '#66b3cc','display': 'inline-block', 'vertical-align': 'middle',
                       'width': '10%', 'height': '7em'}),
        html.Div(style={'width': '10%', 'display': 'inline-block', 'vertical-align': 'middle'})
                 ],
        style={'margin-bottom': '-4em'})
     ,
    # ROUTE DROPDOWN AND DATE SLIDER
    
        # DROPDOWN
    html.Div([html.Div(style={'width': '25%', 'margin-bottom': '1em', 'display': 'inline-block'}),
    
    html.Div([html.Div([html.Label('Origin', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_origin',
    options=[],
    placeholder="Select origin",                                 
    value=sorted(set(checkbox_values[0])),
    multi=True)],
    style={'margin-bottom': '0.5em', 'width': '80%', 'margin-left': '6em'})
    ,
    html.Div([html.Label('Destination', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_destination',
    options=[],
    placeholder="Select destination",                                 
    value=sorted(set(checkbox_values[1])),
    multi=True)],
    style={'margin-bottom': '0.5em', 'width': '80%', 'margin-left': '6em'})
    ,
    html.Div([html.Label('Departure Time', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_deptime',
    options=[],
    placeholder="Select departure time",                                 
    value=sorted(set(checkbox_values[2])),
    multi=True)],
    style={'margin-bottom': '0.5em', 'width': '80%', 'margin-left': '6em'})
    ,          
    html.Div([html.Label('Minimum Stay', style={'textAlign': 'center', 'font-weight': 'bold'}), 
    dcc.Dropdown(id = 'dropdown_stay',
    options=[],
    placeholder="Select duration",                                 
    value=sorted(set(checkbox_values[3])),
    multi=True)],
    style={'margin-bottom': '0.5em', 'width': '80%', 'margin-left': '6em'})
             ],
    style={'width': '50%', 'margin-bottom': '1em', 'margin-top': '0','display': 'inline-block', 'background-color': 'lightgrey'}),
    
    html.Div(style={'width': '25%', 'margin-bottom': '0.5em', 'margin-top': '-3em', 'display': 'inline-block'})
             ]),
    # GRAPH CONTAINER                   
    html.Div(children=[dcc.Graph(
                    id='graph')],
    style={'width': '45%', 'float': 'right', 'display': 'inline-block', 'margin-top': '0.5em'})
        ])

In [ ]:
@app.callback(
    Output("dropdown_origin", "options"),
    [Input("dropdown_destination", "value"),
    Input("dropdown_deptime", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_route(values_destination,values_deptime,values_stay):
    subset=set([checkbox_values[0][i] for i in range(len(checkbox_values[0]))\
    if checkbox_values[1][i] in values_destination\
    and checkbox_values[2][i] in values_deptime\
    and checkbox_values[3][i] in values_stay])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    Output("dropdown_destination", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_deptime", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_route(values_origin,values_deptime,values_stay):
    subset=set([checkbox_values[1][i] for i in range(len(checkbox_values[1]))\
    if checkbox_values[0][i] in values_origin\
    and checkbox_values[2][i] in values_deptime\
    and checkbox_values[3][i] in values_stay])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    Output("dropdown_deptime", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"),
    Input("dropdown_stay", "value")]
    )
def set_checklist_route(values_origin,values_destination,values_stay):
    subset=set([checkbox_values[2][i] for i in range(len(checkbox_values[2]))\
    if checkbox_values[0][i] in values_origin\
    and checkbox_values[1][i] in values_destination\
    and checkbox_values[3][i] in values_stay])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

##########################################################

@app.callback(
    Output("dropdown_stay", "options"),
    [Input("dropdown_origin", "value"),
    Input("dropdown_destination", "value"),
    Input("dropdown_deptime", "value")]
    )
def set_checklist_route(values_origin,values_destination,values_deptime):
    subset=set([checkbox_values[3][i] for i in range(len(checkbox_values[3]))\
    if checkbox_values[0][i] in values_origin\
    and checkbox_values[1][i] in values_destination\
    and checkbox_values[2][i] in values_deptime])
    values=[{'label': i,'value': i} for i in sorted(subset)]
    return values

In [ ]:
@app.callback(Output('graph', 'figure'),
     #,
     # Output('graph', 'figure')
    [Input('dropdown_origin', 'value'),
    Input('dropdown_destination', 'value'),
    Input('dropdown_deptime', 'value'),
    Input('dropdown_stay', 'value')]
    )
def update_plot(values_origin, values_destination, values_deptime, values_stay):
    
    dff_plot = Data.loc[Data['origin'].apply(lambda x: x in values_origin)\
                   &Data['destination'].apply(lambda x: x in values_destination)\
                   &Data['outbound_departure_time'].apply(lambda x: x in values_deptime)\
                   &Data['min_stay'].apply(lambda x: x in values_stay)\
                   &(Data['carrier']=='LG'),:].sort_values(by=['outbound_departure_date'])
    
    traces = []
        
    for i in ['price_outbound_min','price_inbound_min','price_exc_min']:
        traces.append(go.Scatter(
           x=dff_plot['outbound_departure_date'].tolist(),
           y=dff_plot[i].tolist(),
           mode = 'lines',
           name = i
           #,
           #line = dict(
           #dash = linedash_dict_Bid[i],
           #color = linecolor_dict_Bid[i],
           #width = 2
             )
        )
   
        return {'data': traces, 'layout': 
        go.Layout(title=go.layout.Title(
        text='tst',
        xref='paper',
        x=0.5),
        #xaxis={'title': 'Days prior to departure', 'range': [0, 365]},
        #yaxis={'title': 'Bid Price', 'range': [0,dff_plot['price_exc_min'].max()+5]},
        height=600)}
    else:   
        return {'data': []}         

In [ ]:
if __name__ == '__main__':
    #app.run_server(debug=False,host='10.0.0.8',port=port[analyst])
    app.run_server(debug=False,host='localhost',port=4000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jun/2019 18:32:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "GET /assets/Luxair.jpg HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:27] "GET /assets/LuxairGroup.jpg HTTP/1.1" 404 -
127.0.0.1 - - [24/Jun/2019 18:32:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:32:30] "POST /_dash-update-component HTTP/1

127.0.0.1 - - [24/Jun/2019 18:33:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2019 18:33:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.